<a href="https://colab.research.google.com/github/parth-pai/Learners_Space_2023_NLP/blob/main/LS_Training_Model_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jul 15 13:15:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

After giving GPU access, we download necessary libraries like :


1.   `transformers` for importing the pipelines and for tokenizer
2.   `accelerate` for faster loading.
3.   `sentencepiece` while applying the pipeline.
4.   `gradio` for interactive app experience
5.   `datasets` to import datasets
6.   `evaluate` for loading the metric and evaluating
7.   `sacrebleu` for the metric


In [2]:
! pip install -q transformers accelerate sentencepiece gradio datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6

In [3]:
from datasets import load_dataset
raw_dataset = load_dataset("enimai/MuST-C-de")
raw_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/229703 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1423 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2641 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 229703
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1423
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 2641
    })
})

Splitting the dataset with a ratio of 80% in training dataset and 20% in testing dataset with seed value.

In [4]:
from sklearn.model_selection import train_test_split
split_dataset = raw_dataset["train"].train_test_split(test_size=0.2, seed=30)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 183762
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 45941
    })
})

Pre-process the text to give tokenized text

In [5]:
def pre_process_text(text):
  inputs = []
  for sample in text['en']:
    inputs.append(sample)

  outputs = []
  for sample in text['de']:
    outputs.append(sample)

  tokenized_text=tokenizer(inputs, text_target=outputs, max_length=200)
  return tokenized_text

In [6]:
from transformers import pipeline
model_name="Helsinki-NLP/opus-mt-en-de"
translator=pipeline("translation", model=model_name)
translator("The buildings in that city look so organised and beautiful. A perfect concrete jungle.")

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Die Gebäude in dieser Stadt sehen so organisiert und schön aus. Ein perfekter Betondschungel.'}]

`AutoTokenizer` is used to load tokenizer values for a wide range of models. Here it is used to automatically select appropriate tokens based on model's architecture.

In [7]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name, return_tensors="pt")

Gives the tokenised dataset with preprocesing and removing the columns like `['en']`,`['it']` etc

In [8]:
tokenized_dataset=split_dataset.map(
    pre_process_text,
    batched=True,
    remove_columns=split_dataset['train'].column_names
)

Map:   0%|          | 0/183762 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/45941 [00:00<?, ? examples/s]

In the below given code:

1.   class `AutoModelForSeq2SeqLM` loads a pre-trained sequence-to-sequence language model
2.   A data collator `DataCollatorForSeq2Seq` is used to preprocess and format data for input to the model during training or evaluation



In [9]:
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

model= AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [10]:
batches = data_collator([tokenized_dataset["train"][i] for i in range(1,3)])
batches['decoder_input_ids']

tensor([[58100,   302,    65,   324, 16687,    80,     2,   334,    84,  1981,
          2530,  3982,    10,   134,    11, 22087,  1032,   214,   107,   106,
         37306,    17, 15054,   807,     3],
        [58100,   105, 30916,    65,   174,  1292,     3,     0, 58100, 58100,
         58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
         58100, 58100, 58100, 58100, 58100]])

##**Evaluating the Accuracy**
Here we are evaluating our model using sacrebleu score. It checks for matching words and gives lesser score if they aren't matching.

In [11]:
import evaluate
metric_eval = evaluate.load("sacrebleu")

Here we compute the metrics by considering the predictions and lables etc. And get the score out as return

In [12]:
import numpy as np

def compute_metrics(eval):
  preds, label = eval
  if isinstance(preds, tuple):
    preds=preds[0]
  decoded_preds= tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels=np.where(labels !=100, labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.batch_decode(labels, skip_special_tokens=True)

  decoded_preds=[pred.strip() for pred in decoded_preds]
  decoded_labels=[[label.strip()] for label in decoded_labels]

  result=metric_eval.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}

Now we will log into Hugging Face notebook

In [16]:
from huggingface_hub import notebook_login
notebook_login()

##**Fine-Tuning**
Now we use `Seq2SeqTrainingArguments` class which provides a way to specify various training settings and hyperparameters for seq2seq models

In [17]:
from transformers import Seq2SeqTrainingArguments

arg= Seq2SeqTrainingArguments(
    f"model-en-to-de",
    evaluation_strategy="no",
    save_strategy="epoch",
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We now use the `Seq2SeqTrainer` instance used to train and evaluate sequence to sequence model. We can start the training after initializing it.

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    arg,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Step,Training Loss
500,1.458900
1000,1.442400
1500,1.437800
2000,1.423000
2500,1.415200
3000,1.406600
3500,1.414400
4000,1.412900
4500,1.402100
5000,1.403300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=17229, training_loss=1.3008796168920769, metrics={'train_runtime': 3877.9096, 'train_samples_per_second': 142.161, 'train_steps_per_second': 4.443, 'total_flos': 1.0982036336541696e+16, 'train_loss': 1.3008796168920769, 'epoch': 3.0})

In [21]:
from google.colab import drive
drive.mount('/content/drive')